In [1]:
import sys

sys.path.append("../src")

from fabricengineer.import_module.import_module import import_module

VERSION = "0.0.9"

In [2]:
import os
from pyspark.sql import SparkSession


class NotebookUtilsFSMock:
    def _get_path(self, file: str) -> str:
        return os.path.join(os.getcwd(), file)

    def exists(self, path: str) -> bool:
        return os.path.exists(self._get_path(path))

    def put(
        self,
        file: str,
        content: str,
        overwrite: bool = False
    ) -> None:
        path = self._get_path(file)
        os.makedirs(os.path.dirname(path), exist_ok=True)

        if os.path.exists(path) and not overwrite:
            raise FileExistsError(f"File {path} already exists and overwrite is set to False.")
        with open(path, 'w') as f:
            f.write(content)


class NotebookUtilsMock:
    def __init__(self):
        self.fs = NotebookUtilsFSMock()

global spark
spark: SparkSession = SparkSession.builder.appName("PlaygroundSparkSession").getOrCreate()

global notebookutils
notebookutils = NotebookUtilsMock()

25/08/05 13:03:35 WARN Utils: Your hostname, MacBook-Air-von-Enrico.local resolves to a loopback address: 127.0.0.1; using 192.168.0.7 instead (on interface en0)
25/08/05 13:03:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/05 13:03:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
code = import_module("transform.silver.insertonly", VERSION)

print(code)

import os
import logging

from datetime import datetime
from typing import Callable
from abc import ABC, abstractmethod
from dataclasses import dataclass
from uuid import uuid4

from pyspark.sql import (
    SparkSession,
    DataFrame,
    functions as F,
    types as T,
    Window
)


logging.getLogger("py4j").handlers.clear()

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] [%(levelname)s] %(name)s %(message)s",
    datefmt="%d.%m.%Y %H:%M:%S"
)

logger = logging.getLogger("fabricengineer")


@dataclass
class LakehouseTable:
    lakehouse: str
    schema: str
    table: str

    @property
    def table_path(self) -> str:
        return f"{self.lakehouse}.{self.schema}.{self.table}"


@F.udf(returnType=T.StringType())
def generate_uuid():
    """Generate a UUID4 for spark column."""
    return str(uuid4())


@dataclass(frozen=True)
class ConstantColumn:
    """Class for adding a column with constant value to etl"""
    name: str
    value: str
    part_of_nk: b

In [ ]:
# with open("./code.py", "w") as f:
#     f.write(code)

In [5]:
exec(code)

In [6]:
etl

In [7]:
table = LakehouseTable(
    lakehouse="test_lakehouse",
    schema="test_schema",
    table="test_table"
)
table

LakehouseTable(lakehouse='test_lakehouse', schema='test_schema', table='test_table')

In [8]:
code = import_module("transform.silver.mlv", VERSION)

print(code, globals())

from typing import Any
from pyspark.sql import DataFrame, SparkSession
from fabricengineer.logging.logger import logger


def to_spark_sql(sql: str) -> str:
    return sql \
            .replace("[", "`") \
            .replace("]", "`")


class MaterializedLakeView:
    def __init__(
        self,
        lakehouse: str = None,
        schema: str = None,
        table: str = None,
        table_suffix: str = "_mlv",
        spark_: SparkSession = None,
        notebookutils_: Any = None,
        is_testing_mock: bool = False
    ) -> None:
        self.init(
            lakehouse=lakehouse,
            schema=schema,
            table=table,
            table_suffix=table_suffix,
            spark_=spark_,
            notebookutils_=notebookutils_,
            is_testing_mock=is_testing_mock
        )

    def init(
        self,
        lakehouse: str,
        schema: str,
        table: str,
        table_suffix: str = "_mlv",
        spark_: SparkSession = None,
        notebookut

In [9]:
exec(code, globals())

In [11]:
mlv.init(
    lakehouse="TestLakehouse",
    schema="schema",
    table="table",
    is_testing_mock=True
)

sql = "SELECT * FROM TestLakehouse.schema.table2"
mlv.create_or_replace(sql, mock_is_existing=False)

[05.08.2025 13:04:02] [INFO] fabricengineer REPLACE MLV: TestLakehouse.schema.table_mlv
[05.08.2025 13:04:02] [INFO] fabricengineer DROP MATERIALIZED LAKE VIEW IF EXISTS TestLakehouse.schema.table_mlv
[05.08.2025 13:04:02] [INFO] fabricengineer CREATE SCHEMA IF NOT EXISTS TestLakehouse.schema
[05.08.2025 13:04:02] [INFO] fabricengineer CREATE MLV: TestLakehouse.schema.table_mlv
